In [3]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")


In [4]:
from langchain_groq import ChatGroq

In [5]:
llm= ChatGroq(model="llama3-8b-8192")

In [6]:
llm.invoke("what is triangular ineuqality theorem")

AIMessage(content='A fundamental theorem in mathematics!\n\nThe Triangular Inequality Theorem, also known as the Triangle Inequality, is a fundamental property of metric spaces (e.g., Euclidean spaces) that states:\n\n**For any two points A and B in a metric space, the following holds:**\n\n|AB| ≤ |AC| + |CB|\n\nwhere:\n\n* |AB| is the distance between points A and B\n* |AC| is the distance between points A and C\n* |CB| is the distance between points C and B\n\nIn other words, the distance between two points is less than or equal to the sum of the distances between each point and a third point, which is called the "triangle inequality".\n\nThis theorem has far-reaching implications in various areas of mathematics, such as:\n\n1. **Geometry**: It helps to establish the properties of triangles, including the fact that the sum of the lengths of any two sides is greater than or equal to the length of the third side (the "triangle inequality" in the classical sense).\n2. **Analysis**: It i

In [7]:
from langchain_core.tools import tool

In [8]:
@tool
def add(a: int, b : int) -> int:
    """Adds a and b."""
    return a+b
@tool
def multiply(a: int, b: int) -> int:
    """multiplies a and b."""
    return a*b

tools=[add,multiply]

In [9]:
llm_with_tools=llm.bind_tools(tools)

In [10]:
query ="what is 120*5"

In [11]:
async for chunk in llm_with_tools.astream(query):
    print(chunk.tool_call_chunks)

[]
[{'name': 'multiply', 'args': '{"a":120,"b":5}', 'id': 'call_rw2e', 'index': 0, 'type': 'tool_call_chunk'}]
[]


In [12]:
first = True
async for chunk in llm_with_tools.astream(query):
    if first:
        gathered = chunk
        first = False
    else:
        gathered = gathered + chunk

    print(gathered.tool_call_chunks)

[]
[{'name': 'multiply', 'args': '{"a":120,"b":5}', 'id': 'call_4z78', 'index': 0, 'type': 'tool_call_chunk'}]
[{'name': 'multiply', 'args': '{"a":120,"b":5}', 'id': 'call_4z78', 'index': 0, 'type': 'tool_call_chunk'}]


In [13]:
llm_with_tools.invoke(
query).tool_calls

[{'name': 'multiply',
  'args': {'a': 120, 'b': 5},
  'id': 'call_2vyr',
  'type': 'tool_call'}]

In [16]:
query="Whats 119 times 8 plus 20. Don't do any math yourself, only use tools for math. Respect order of operations"


In [17]:
llm_with_tools.invoke(
query).tool_calls

[{'name': 'multiply',
  'args': {'a': 119, 'b': 8},
  'id': 'call_qaz2',
  'type': 'tool_call'}]

The model shouldn't be trying to add anything yet, since it technically can't know the results of 119 * 8 yet.

By adding a prompt with some examples 
# few shots for tools usage
 we can correct this behavior:

In [20]:
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

examples = [
    HumanMessage(
        "What's the product of 317253 and 128472 plus four", name="example_user"
    ),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[
            {"name": "Multiply", "args": {"x": 317253, "y": 128472}, "id": "1"}
        ],
    ),
    ToolMessage("16505054784", tool_call_id="1"),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[{"name": "Add", "args": {"x": 16505054784, "y": 4}, "id": "2"}],
    ),
    ToolMessage("16505054788", tool_call_id="2"),
    AIMessage(
        "The product of 317253 and 128472 plus four is 16505054788",
        name="example_assistant",
    ),
]

system = """You are bad at math but are an expert at using a calculator. 

Use past tool usage as an example of how to correctly use the tools."""
few_shot_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        *examples,
        ("human", "{query}"),
    ]
)

chain = {"query": RunnablePassthrough()} | few_shot_prompt | llm_with_tools
chain.invoke("Whats 119 times 8 plus 20").tool_calls

In [21]:


model_with_tools = llm.bind(
    tools=[
        {
            "type": "function",
            "function": {
                "name": "multiply",
                "description": "Multiply two integers together.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "a": {"type": "number", "description": "First integer"},
                        "b": {"type": "number", "description": "Second integer"},
                    },
                    "required": ["a", "b"],
                },
            },
        }
    ]
)

model_with_tools.invoke("Whats 119 times 8?")

AIMessage(content='/tool-use>\n{\n\t"tool_calls": [\n\t\t{\n\t\t\t"id": "pending",\n\t\t\t"type": "function",\n\t\t\t"function": {\n\t\t\t\t"name": "multiply"\n\t\t\t},\n\t\t\t"parameters": {\n\t\t\t\t"a": 119,\n\t\t\t\t"b": 8\n\t\t\t}\n\t\t}\n\t]\n}\n</tool-use>', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 78, 'prompt_tokens': 938, 'total_tokens': 1016, 'completion_time': 0.065, 'prompt_time': 0.113092944, 'queue_time': 0.019255473999999995, 'total_time': 0.178092944}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'stop', 'logprobs': None}, id='run-78658c96-3730-46f7-be9b-4e69ff93884d-0', usage_metadata={'input_tokens': 938, 'output_tokens': 78, 'total_tokens': 1016})

### we can force our tool to call the multiply tool by using the following code:

In [22]:
llm_forced_to_multiply = llm.bind_tools(tools, tool_choice="multiply")
llm_forced_to_multiply.invoke("what is 2 + 4")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_rbhf', 'function': {'arguments': '{"a":2,"b":4}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 79, 'prompt_tokens': 1892, 'total_tokens': 1971, 'completion_time': 0.065833333, 'prompt_time': 0.227822282, 'queue_time': -0.287948732, 'total_time': 0.293655615}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-2b37675a-0d06-45f5-b6d3-a42475bb0813-0', tool_calls=[{'name': 'multiply', 'args': {'a': 2, 'b': 4}, 'id': 'call_rbhf', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1892, 'output_tokens': 79, 'total_tokens': 1971})

In [24]:
llm_forced_to_use_tool = llm.bind_tools(tools, tool_choice="any")


In [27]:
#llm_forced_to_use_tool.invoke("What IS TODAY?")